# Crop Segmentation and Classification

In this notebook, we demonstrate and compare techniques for segmentation and classification of crops using Planet imagery. A metric will be established to quantify accuracy of crop classification and will be used to compare technique results.

The general sections are:
1. [Identify area of interest and data](#data)
2. [Crop Segmentation](#segmentation)
3. [Crop Classification](#classification)

In [1]:
# Notebook dependencies
from __future__ import print_function

from functools import partial
import json
import os


import ipyleaflet as ipyl
import ipywidgets as ipyw
import fiona
import pyproj
import shapely
from shapely.geometry import shape, mapping
from shapely.ops import transform

<a id='data'></a>

## Identify area of interest and data

Ground truth crop type and boundary data is not easy to come by. Therefore, the area and time of interest for this problem is primarily defined by the availability of ground truth data. The [2015 Sacramento County Land Use DWR Survey Dataset](http://www.water.ca.gov/landwateruse/lusrvymain.cfm) is a free dataset covering Sacramento county in 2015. It provides hand-adjusted boundaries and provides crop types.

The satellite imagery we will use in this study are Landsat 8 and SSO Planetscope 2 imagery. SSO Planetscope 2 satellites were launched Feb 14, 2017 ([news release](https://www.planet.com/pulse/planet-launches-satellite-constellation-to-image-the-whole-planet-daily/)), therefore they did not image Sacramento county in 2015. Landsat 8 will be used as a bridge between 2015 and 2017.

### Explore Ground Truth Data

#### Downloading data
To obtain and prepare the ground truth data, download the shapefile zip file from (http://www.water.ca.gov/landwateruse/docs/landusedata/shapes/15sa.zip) and then unzip into /data.

Execute the following commands in `crop-classification` directory:
```
mkdir data
mkdir data/dwr_survey
wget http://www.water.ca.gov/landwateruse/docs/landusedata/shapes/15sa.zip
unzip 15sa.zip -d data/dwr_survey/
```

expected output:
```
Archive:  15sa.zip
  inflating: data/dwr_survey/SA15.dbf  
  inflating: data/dwr_survey/SA15.jgw  
  inflating: data/dwr_survey/SA15.jpg  
  inflating: data/dwr_survey/SA15.prj  
  inflating: data/dwr_survey/SA15.shp  
  inflating: data/dwr_survey/SA15.shp.xml  
  inflating: data/dwr_survey/SA15.shx  
  inflating: data/dwr_survey/DWR Data Disclaimer.pdf  
  inflating: data/dwr_survey/SA15Meta.pdf  
  inflating: data/dwr_survey/attributes_rev02102010.pdf  
  inflating: data/dwr_survey/09legend.pdf  
```

In [2]:
# Let's specify the shapefile location and ensure it indeed exists
survey_shapefile = 'data/dwr_survey/SA15.shp'
assert os.path.isfile(survey_shapefile)

#### Load the survey shapefile

We will use [fiona](http://toblerity.org/fiona/manual.html) to load the shapefile, [shapely](http://toblerity.org/shapely/manual.html) to manage the geometries

##### Reproject to WGS84

What is the coordinate reference system for this dataset?

In [3]:
with fiona.open(survey_shapefile, 'r') as survey_data:
    src_proj = survey_data.crs['init']
print(src_proj)

epsg:26910


Turns out it is [EPSG:26910](http://spatialreference.org/ref/epsg/26910/). Geojson only supports [EPSG:4326](http://spatialreference.org/ref/epsg/4326/). We will need to reproject the shapes.

In [15]:
# define projection
# from shapely [docs](http://toblerity.org/shapely/manual.html#shapely.ops.transform)
dst_proj = 'epsg:4326'
project_to_wkt = partial(
    pyproj.transform,
    pyproj.Proj(init=src_proj),
    pyproj.Proj(init=dst_proj))

def project_feature(feat):
    g1 = shape(feat['geometry'])
    g2 = transform(project_to_wkt, g1)
    feat['geometry'] = mapping(g2)

##### Filter survey data to agricultural classes

The survey data has attributes that provide the crop type. These attributes are described in a pdf distributed with the shapefile. It was unzipped along with the shapefile files and is located at `data/dwr_survey/09legend.pdf`.

We are interested in the agricultural classes. Class is specified by the 'CLASS1' attribute of the feature and agricultural classes are: G, R, F, P, T, D, C, and V.

In [16]:
agg_classes = ['G', 'R', 'F', 'P', 'T', 'D', 'C', 'V']

def is_agricultural(feat):
    return feat['properties']['CLASS1'] in agg_classes

#### Finally: Load data

In [17]:
features = []
with fiona.open(survey_shapefile) as survey_data:
    for feat in survey_data:
        if is_agricultural(feat):
            project_feature(feat)
            features.append(feat)
print(len(features))

7429


In [18]:
# Assign colors to classes
# colors determined using [colorbrewer2.org](http://colorbrewer2.org/#type=sequential&scheme=BuGn&n=3)
colors = ['#ffffd9','#edf8b1','#c7e9b4','#7fcdbb','#41b6c4','#1d91c0','#225ea8','#0c2c84']
class_colors = dict((a,c) for a,c in zip(agg_classes, colors))
def get_color(cls):
    return class_colors[cls]

In [21]:
# Initialize map, deleting map instance if it exists
try:
    del filtered_map
except NameError:
    pass

zoom = 9
center = [38.3586252, -121.3853994] # lat/lon
bounds = None #[(38.23467444211417, -121.66706085205077), (38.342464018725714, -121.32030487060547)]

In [22]:
# Reuse map parameters if map already defined
try:
    center = filtered_map.center
    print(center)
    zoom = filtered_map.zoom
    print(zoom)
    bounds = filtered_map.bounds
    print(bounds)
except NameError:
    pass

# Create the map
filtered_map = ipyl.Map(
        center=center, 
        zoom=zoom,
    )

if bounds is not None:
    # create bound box geometry from map bounds readout
    bounds_box = shapely.geometry.box(bounds[0][1], bounds[0][0],
                                      bounds[1][1], bounds[1][0])

    # display bounds box
    bounds_feature = {
        'geometry':mapping(bounds_box),
        "properties":{
                'style':{'color': 'gray','fillColor': 'gray','fillOpacity': 0.5,'weight': 1}},
        'type':u"Feature"}

    bounds_layer = ipyl.GeoJSON(data=bounds_feature)
    filtered_map.add_layer(bounds_layer)

    # filter to bounds
    display_features = [f for f in agg_features if shape(f['geometry']).within(bounds_box)]
    
else:
    display_features = agg_features

feature_collection = {
    "type": "FeatureCollection",
    "features": display_features
}

for f in feature_collection['features']:
    feature_color = get_color(f['properties']['CLASS1'])
    f['properties']['style'] = {
        'weight': 0,
        'fillColor': feature_color,
        'fillOpacity': 1}

feature_layer = ipyl.GeoJSON(data=feature_collection)

# Display class as label
label = ipyw.Label(layout=ipyw.Layout(width='100%'))
def hover_handler(event=None, id=None, properties=None):
    label.value = 'class: {}'.format(properties['CLASS1'])

feature_layer.on_hover(hover_handler)
filtered_map.add_layer(feature_layer)   

# Display
ipyw.VBox([filtered_map, label])

# NOTE: it may take a little while for this to display

<a id='segmentation'></a>

## Crop segmentation

TODO

<a id='classification'></a>

## Crop classification

TODO